# Digit Eye


In [1]:
%load_ext autoreload
%autoreload 2
import torch

from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset
from typing import cast

from src.models import Cifar10Model

torch.manual_seed(1)

dataset_id = "cifar10"
def load_torch_dataset(split: str):
    dataset = load_dataset(dataset_id, split = split).with_format("torch") # type: ignore
    return cast(Dataset, dataset)

train_set = load_torch_dataset("train")
test_set = load_torch_dataset("test")

def collate_fn(batches):
    features = torch.stack([batch["img"] for batch in batches])
    labels = torch.tensor([batch["label"] for batch in batches])
    return features.float(), labels.long()

batch_size = 128
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)


model_path = "./assets/digit_eye"
model = Cifar10Model(path=model_path)

model.train(epochs=10, train_loader=train_loader, test_loader=test_loader)
model.load_weights()
model.evaluate(test_loader)


/home/anamnesis/Documents/UR/Aprendizaje Automático de Máquina/Proyecto Final/.venv/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset cifar10 (/home/anamnesis/.cache/huggingface/datasets/cifar10/plain_text/1.0.0/447d6ec4733dddd1ce3bb577c7166b986eaa4c538dcd9e805ba61f35674a9de4)
Found cached dataset cifar10 (/home/anamnesis/.cache/huggingface/datasets/cifar10/plain_text/1.0.0/447d6ec4733dddd1ce3bb577c7166b986eaa4c538dcd9e805ba61f35674a9de4)



N° Epoch 0
Trainning loss: 0.013857526664733886	Accuracy: 0.38161998987197876
Test loss: 0.015020396995544433	Accuracy: 0.3555000126361847
Best Accuracy Found: -inf -> 0.3555000126361847. Saving Model...

N° Epoch 1
Trainning loss: 0.012246523303985596	Accuracy: 0.4535999894142151
Test loss: 0.013573711752891541	Accuracy: 0.41100001335144043
Best Accuracy Found: 0.3555000126361847 -> 0.41100001335144043. Saving Model...

N° Epoch 2
Trainning loss: 0.011494333777427673	Accuracy: 0.48767998814582825
Test loss: 0.01151046404838562	Accuracy: 0.4814000129699707
Best Accuracy Found: 0.41100001335144043 -> 0.4814000129699707. Saving Model...

N° Epoch 3
Trainning loss: 0.01099975510597229	Accuracy: 0.5074599981307983
Test loss: 0.012609583389759064	Accuracy: 0.4514999985694885

N° Epoch 4
Trainning loss: 0.010704677309989929	Accuracy: 0.5217800140380859
Test loss: 0.013580440509319305	Accuracy: 0.4196999967098236

N° Epoch 5
Trainning loss: 0.010487741627693177	Accuracy: 0.5311200022697449
T